# Prepare data for Pipeline

## Setup and settings

In [1]:
# Importing packages
import os
import yaml
import logging
import pandas as pd
from Bio.Seq import Seq
from qiime2 import Artifact, Visualization
from qiime2.plugins import demux
from qiime2.plugins.cutadapt.methods import trim_paired

### Receiving the parameters

The following cell can receive parameters using the [papermill](https://papermill.readthedocs.io/en/latest/) tool.

In [2]:
params_path = os.path.join('..', 'params', 'ana-flavia-superlactacao.yaml')
experiment_name = 'jenneffer-vs-01'
base_dir = os.path.join('/', 'home', 'lauro', 'nupeb', 'redemicro')
manifest_file = os.path.join(base_dir, 'data', 'raw', 'manifest', 'not-hist-vs-manifest.csv')
img_folder = os.path.abspath(os.path.join(base_dir, 'imgs'))
replace_files = False
trim = None

In [3]:
# Parameters
experiment_name = "ana-flavia-NCxHSD-NC-trim"
base_dir = "/home/lauro/nupeb/rede-micro/redemicro-ana-flavia-nutri"
manifest_file = "/home/lauro/nupeb/rede-micro/redemicro-ana-flavia-nutri/data/raw/manifest/manifest-ana-flavia-NCxHSD-NC.csv"
metadata_file = "/home/lauro/nupeb/rede-micro/redemicro-ana-flavia-nutri/data/raw/metadata/metadata-ana-flavia-NCxHSD-NC.tsv"
class_col = "group-id"
classifier_file = "/home/lauro/nupeb/rede-micro/models/silva-138-99-nb-classifier.qza"
top_n = 20
replace_files = False
phred = 20
trunc_f = 0
trunc_r = 0
overlap = 12
threads = 6
trim = {
    "overlap": 8,
    "forward_primer": "CCTACGGGRSGCAGCAG",
    "reverse_primer": "GGACTACHVGGGTWTCTAAT",
}


### Defining names and paths

In [4]:
# Define the output folder path
out_dir = os.path.join(base_dir, 'experiments', experiment_name, 'qiime-artifacts')
img_folder = os.path.abspath(os.path.join(base_dir, 'experiments', experiment_name, 'imgs'))

# Create path if it not exists
if not os.path.isdir(out_dir):
    os.makedirs(out_dir)
    print(f'New qiime-artifacts folder path created: {out_dir}')

# Create path if it not exists
if not os.path.isdir(img_folder):
    os.makedirs(img_folder)
    print(f'New img folder path created: {img_folder}')

# Define the output artifact full path
demux_file = os.path.join(out_dir, 'demux-paired.qza')
demux_view = os.path.join(out_dir, 'demux-paired.qzv')
demux_file_trim = os.path.join(out_dir, 'demux-paired-trim.qza')
demux_view_trim = os.path.join(out_dir, 'demux-paired-trim.qzv')

New img folder path created: /home/lauro/nupeb/rede-micro/redemicro-ana-flavia-nutri/experiments/ana-flavia-NCxHSD-NC-trim/imgs


## Step execution

This Step import all `fastq` files in a **QIIME2 Artifact** object and save it to a new `qza` file of `SampleData[PairedEndSequencesWithQuality]`

In [5]:
manifest_df = pd.read_csv(manifest_file)
n_directions = len(manifest_df['direction'].unique())
if n_directions == 1:
    d_type = 'SampleData[SequencesWithQuality]'
    v_type = 'SingleEndFastqManifestPhred33'
elif n_directions == 2:
    d_type = 'SampleData[PairedEndSequencesWithQuality]'
    v_type = 'PairedEndFastqManifestPhred33'
else:
    print(f'ERROR: invalid number of directions {n_directions}')

In [6]:
# If the demux file does not exist or if the old file will be replaced
if not os.path.isfile(demux_file) or replace_files:
    
    # Import data and create an artifact object
    artifact = Artifact.import_data(
        d_type, 
        manifest_file, 
        view_type=v_type)
    
    # Save the artifact object to a new qza file
    artifact.save(demux_file)

else:
    artifact = Artifact.load(demux_file)

In [7]:
if not os.path.isfile(demux_view) or replace_files: 
    # Generate e visualization of the Artifact
    demux_summary = demux.visualizers.summarize(artifact)

    # Save a new visualization file based on the qza file
    Visualization.save(demux_summary.visualization, filepath=demux_view)
    
    demux_view_obj = demux_summary.visualization
else:
    demux_view_obj = Visualization.load(demux_view)

## Step report


In [8]:
print(demux_view_obj)

<visualization: Visualization uuid: 31b0eebd-011d-47c5-a5da-ce3d674e7e73>


In [9]:
# Render Visualization
demux_view_obj

<visualization: Visualization uuid: 31b0eebd-011d-47c5-a5da-ce3d674e7e73>

[cutadapt](https://docs.qiime2.org/2022.2/plugins/available/cutadapt/trim-paired/)

We trim the forward primer and the reverse complement of the reverse primer from the forward reads. We trim the reverse primer and reverse complement of the forward primer from the reverse reads.

In [10]:
demux_file_trim = os.path.join(out_dir, 'demux-paired-trim.qza')
demux_view_trim = os.path.join(out_dir, 'demux-paired-trim.qzv')

if trim and (not os.path.isfile(demux_file_trim) or replace_files):
    forward_primer = [trim['forward_primer']] # ['CCTACGGGRSGCAGCAG']
    reverse_primer = [trim['reverse_primer']] # ['GGACTACHVGGGTWTCTAAT']
    forward_reverse_complement = [str(Seq(forward_primer[0]).reverse_complement())]
    reverse_reverse_complement = [str(Seq(reverse_primer[0]).reverse_complement())]

    res = trim_paired(
        demultiplexed_sequences=artifact,
        front_f=forward_primer,
        front_r=reverse_primer,
#         adapter_f=reverse_reverse_complement,
#         adapter_r=forward_reverse_complement,
        cores=threads,
        overlap=trim['overlap'],
        indels=False,
        match_read_wildcards=True,
        match_adapter_wildcards=True,
        error_rate=0.01,
        discard_untrimmed=True,
    ).trimmed_sequences

    demux.visualizers.summarize(res).visualization
    res.save(demux_file_trim)
    Visualization.save(demux.visualizers.summarize(res).visualization, filepath=demux_view_trim)

Running external command line application. This may print messages to stdout and/or stderr.
The commands to be run are below. These commands cannot be manually re-run as they will depend on temporary files that no longer exist.

Command: cutadapt --cores 6 --error-rate 0.01 --times 1 --overlap 8 --minimum-length 1 -o /tmp/q2-CasavaOneEightSingleLanePerSampleDirFmt-uyaazuch/210421121673_0_L001_R1_001.fastq.gz -p /tmp/q2-CasavaOneEightSingleLanePerSampleDirFmt-uyaazuch/210421121673_1_L001_R2_001.fastq.gz --front CCTACGGGRSGCAGCAG -G GGACTACHVGGGTWTCTAAT --no-indels --match-read-wildcards --discard-untrimmed /tmp/qiime2-archive-cvpprkbi/714075d8-9cee-47f6-8a87-959a28f4e684/data/210421121673_0_L001_R1_001.fastq.gz /tmp/qiime2-archive-cvpprkbi/714075d8-9cee-47f6-8a87-959a28f4e684/data/210421121673_1_L001_R2_001.fastq.gz



This is cutadapt 3.5 with Python 3.8.12
Command line parameters: --cores 6 --error-rate 0.01 --times 1 --overlap 8 --minimum-length 1 -o /tmp/q2-CasavaOneEightSingleLanePerSampleDirFmt-uyaazuch/210421121673_0_L001_R1_001.fastq.gz -p /tmp/q2-CasavaOneEightSingleLanePerSampleDirFmt-uyaazuch/210421121673_1_L001_R2_001.fastq.gz --front CCTACGGGRSGCAGCAG -G GGACTACHVGGGTWTCTAAT --no-indels --match-read-wildcards --discard-untrimmed /tmp/qiime2-archive-cvpprkbi/714075d8-9cee-47f6-8a87-959a28f4e684/data/210421121673_0_L001_R1_001.fastq.gz /tmp/qiime2-archive-cvpprkbi/714075d8-9cee-47f6-8a87-959a28f4e684/data/210421121673_1_L001_R2_001.fastq.gz
Processing reads on 6 cores in paired-end mode ...


Finished in 3.68 s (10 µs/read; 6.03 M reads/minute).

=== Summary ===

Total read pairs processed:            370,042
  Read 1 with adapter:                 359,997 (97.3%)
  Read 2 with adapter:                 343,009 (92.7%)

== Read fate breakdown ==
Pairs that were too short:                   0 (0.0%)
Pairs discarded as untrimmed:           36,262 (9.8%)
Pairs written (passing filters):       333,780 (90.2%)

Total basepairs processed:   188,721,420 bp
  Read 1:   112,862,810 bp
  Read 2:    75,858,610 bp
Total written (filtered):    155,609,062 bp (82.5%)
  Read 1:    94,978,565 bp
  Read 2:    60,630,497 bp

=== First read: Adapter 1 ===

Sequence: CCTACGGGRSGCAGCAG; Type: regular 5'; Length: 17; Trimmed: 359997 times

Minimum overlap: 8
No. of allowed errors:
1-17 bp: 0

Overview of removed sequences
length	count	expect	max.err	error counts
8	3	5.6	0	3
9	4	1.4	0	4
10	3	0.4	0	3
11	2	0.1	0	2
12	4	0.0	0	4
13	4	0.0	0	4
14	12	0.0	0	12
15	9	0.0	0	9
16	19	0.0	0	19
17	109	0.0	0	109
1

Finished in 4.98 s (9 µs/read; 6.75 M reads/minute).

=== Summary ===

Total read pairs processed:            559,807
  Read 1 with adapter:                 544,656 (97.3%)
  Read 2 with adapter:                 519,894 (92.9%)

== Read fate breakdown ==
Pairs that were too short:                   0 (0.0%)
Pairs discarded as untrimmed:           53,686 (9.6%)
Pairs written (passing filters):       506,121 (90.4%)

Total basepairs processed:   285,501,570 bp
  Read 1:   170,741,135 bp
  Read 2:   114,760,435 bp
Total written (filtered):    235,957,214 bp (82.6%)
  Read 1:   144,019,667 bp
  Read 2:    91,937,547 bp

=== First read: Adapter 1 ===

Sequence: CCTACGGGRSGCAGCAG; Type: regular 5'; Length: 17; Trimmed: 544656 times

Minimum overlap: 8
No. of allowed errors:
1-17 bp: 0

Overview of removed sequences
length	count	expect	max.err	error counts
8	5	8.5	0	5
9	4	2.1	0	4
10	3	0.5	0	3
11	7	0.1	0	7
12	7	0.0	0	7
13	6	0.0	0	6
14	17	0.0	0	17
15	16	0.0	0	16
16	22	0.0	0	22
17	155	0.0	0	155


Finished in 3.21 s (9 µs/read; 6.46 M reads/minute).

=== Summary ===

Total read pairs processed:            345,762
  Read 1 with adapter:                 336,819 (97.4%)
  Read 2 with adapter:                 321,022 (92.8%)

== Read fate breakdown ==
Pairs that were too short:                   0 (0.0%)
Pairs discarded as untrimmed:           32,941 (9.5%)
Pairs written (passing filters):       312,821 (90.5%)

Total basepairs processed:   176,338,620 bp
  Read 1:   105,457,410 bp
  Read 2:    70,881,210 bp
Total written (filtered):    145,841,282 bp (82.7%)
  Read 1:    89,015,404 bp
  Read 2:    56,825,878 bp

=== First read: Adapter 1 ===

Sequence: CCTACGGGRSGCAGCAG; Type: regular 5'; Length: 17; Trimmed: 336819 times

Minimum overlap: 8
No. of allowed errors:
1-17 bp: 0

Overview of removed sequences
length	count	expect	max.err	error counts
8	5	5.3	0	5
9	3	1.3	0	3
10	5	0.3	0	5
11	2	0.1	0	2
12	4	0.0	0	4
13	5	0.0	0	5
14	12	0.0	0	12
15	9	0.0	0	9
16	11	0.0	0	11
17	92	0.0	0	92
18	5

...

Finished in 0.34 s (24 µs/read; 2.52 M reads/minute).

=== Summary ===

Total read pairs processed:             14,319
  Read 1 with adapter:                  13,866 (96.8%)
  Read 2 with adapter:                  12,833 (89.6%)

== Read fate breakdown ==
Pairs that were too short:                   0 (0.0%)
Pairs discarded as untrimmed:            1,875 (13.1%)
Pairs written (passing filters):        12,444 (86.9%)

Total basepairs processed:     7,302,690 bp
  Read 1:     4,367,295 bp
  Read 2:     2,935,395 bp
Total written (filtered):      5,801,125 bp (79.4%)
  Read 1:     3,540,895 bp
  Read 2:     2,260,230 bp

=== First read: Adapter 1 ===

Sequence: CCTACGGGRSGCAGCAG; Type: regular 5'; Length: 17; Trimmed: 13866 times

Minimum overlap: 8
No. of allowed errors:
1-17 bp: 0

Overview of removed sequences
length	count	expect	max.err	error counts
11	1	0.0	0	1
15	1	0.0	0	1
17	5	0.0	0	5
18	2170	0.0	0	2170
19	2437	0.0	0	2437
20	2494	0.0	0	2494
21	2324	0.0	0	2324
22	2426	0.0	0	2426
23	

Finished in 0.43 s (21 µs/read; 2.91 M reads/minute).

=== Summary ===

Total read pairs processed:             20,959
  Read 1 with adapter:                  20,357 (97.1%)
  Read 2 with adapter:                  19,037 (90.8%)

== Read fate breakdown ==
Pairs that were too short:                   0 (0.0%)
Pairs discarded as untrimmed:            2,450 (11.7%)
Pairs written (passing filters):        18,509 (88.3%)

Total basepairs processed:    10,689,090 bp
  Read 1:     6,392,495 bp
  Read 2:     4,296,595 bp
Total written (filtered):      8,628,512 bp (80.7%)
  Read 1:     5,266,608 bp
  Read 2:     3,361,904 bp

=== First read: Adapter 1 ===

Sequence: CCTACGGGRSGCAGCAG; Type: regular 5'; Length: 17; Trimmed: 20357 times

Minimum overlap: 8
No. of allowed errors:
1-17 bp: 0

Overview of removed sequences
length	count	expect	max.err	error counts
15	1	0.0	0	1
16	1	0.0	0	1
17	7	0.0	0	7
18	3085	0.0	0	3085
19	3612	0.0	0	3612
20	3759	0.0	0	3759
21	3463	0.0	0	3463
22	3487	0.0	0	3487
23	

Finished in 0.39 s (19 µs/read; 3.11 M reads/minute).

=== Summary ===

Total read pairs processed:             20,194
  Read 1 with adapter:                  19,610 (97.1%)
  Read 2 with adapter:                  18,045 (89.4%)

== Read fate breakdown ==
Pairs that were too short:                   0 (0.0%)
Pairs discarded as untrimmed:            2,655 (13.1%)
Pairs written (passing filters):        17,539 (86.9%)

Total basepairs processed:    10,298,940 bp
  Read 1:     6,159,170 bp
  Read 2:     4,139,770 bp
Total written (filtered):      8,176,314 bp (79.4%)
  Read 1:     4,990,682 bp
  Read 2:     3,185,632 bp

=== First read: Adapter 1 ===

Sequence: CCTACGGGRSGCAGCAG; Type: regular 5'; Length: 17; Trimmed: 19610 times

Minimum overlap: 8
No. of allowed errors:
1-17 bp: 0

Overview of removed sequences
length	count	expect	max.err	error counts
15	3	0.0	0	3
17	7	0.0	0	7
18	3003	0.0	0	3003
19	3491	0.0	0	3491
20	3617	0.0	0	3617
21	3224	0.0	0	3224
22	3416	0.0	0	3416
23	2844	0.0	0	28

Finished in 0.87 s (13 µs/read; 4.66 M reads/minute).

=== Summary ===

Total read pairs processed:             67,293
  Read 1 with adapter:                  65,501 (97.3%)
  Read 2 with adapter:                  61,930 (92.0%)

== Read fate breakdown ==
Pairs that were too short:                   0 (0.0%)
Pairs discarded as untrimmed:            6,987 (10.4%)
Pairs written (passing filters):        60,306 (89.6%)

Total basepairs processed:    34,319,430 bp
  Read 1:    20,524,365 bp
  Read 2:    13,795,065 bp
Total written (filtered):     28,113,964 bp (81.9%)
  Read 1:    17,159,382 bp
  Read 2:    10,954,582 bp

=== First read: Adapter 1 ===

Sequence: CCTACGGGRSGCAGCAG; Type: regular 5'; Length: 17; Trimmed: 65501 times

Minimum overlap: 8
No. of allowed errors:
1-17 bp: 0

Overview of removed sequences
length	count	expect	max.err	error counts
11	1	0.0	0	1
12	1	0.0	0	1
14	4	0.0	0	4
15	2	0.0	0	2
16	1	0.0	0	1
17	19	0.0	0	19
18	10169	0.0	0	10169
19	11601	0.0	0	11601
20	12117	0.0	0	

Finished in 0.30 s (36 µs/read; 1.65 M reads/minute).

=== Summary ===

Total read pairs processed:              8,334
  Read 1 with adapter:                   8,081 (97.0%)
  Read 2 with adapter:                   7,509 (90.1%)

== Read fate breakdown ==
Pairs that were too short:                   0 (0.0%)
Pairs discarded as untrimmed:            1,046 (12.6%)
Pairs written (passing filters):         7,288 (87.4%)

Total basepairs processed:     4,250,340 bp
  Read 1:     2,541,870 bp
  Read 2:     1,708,470 bp
Total written (filtered):      3,397,413 bp (79.9%)
  Read 1:     2,073,898 bp
  Read 2:     1,323,515 bp

=== First read: Adapter 1 ===

Sequence: CCTACGGGRSGCAGCAG; Type: regular 5'; Length: 17; Trimmed: 8081 times

Minimum overlap: 8
No. of allowed errors:
1-17 bp: 0

Overview of removed sequences
length	count	expect	max.err	error counts
16	1	0.0	0	1
17	3	0.0	0	3
18	1267	0.0	0	1267
19	1465	0.0	0	1465
20	1495	0.0	0	1495
21	1281	0.0	0	1281
22	1390	0.0	0	1390
23	1178	0.0	0	117